In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from datasets import load_dataset
import torch
import os

print("🔧 Starting continued LoRA fine-tuning script...")

# --- Settings ---
base_model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
lora_path = "/kaggle/input/finetuned/tinyllama-lora-finetuned"
dataset_path = "/kaggle/input/medidata"
dataset_file = "second_half.json"
output_dir = "/kaggle/working/tinyllama-lora-finetuned-v2"

print(f"📁 Base Model: {base_model_name}")
print(f"🔁 Continue LoRA fine-tuning from: {lora_path}")
print(f"📂 Dataset path: {os.path.join(dataset_path, dataset_file)}")
print(f"💾 Output will be saved to: {output_dir}")

# --- Load tokenizer ---
print("📦 Step 1: Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer loaded.")

# --- Load base model ---
print("📦 Step 2: Loading base model in 4-bit...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)
print("✅ Base model loaded.")

# --- Prepare model for further LoRA training ---
print("🛠 Step 3: Preparing model for LoRA stacking...")
base_model = prepare_model_for_kbit_training(base_model)

print("🔌 Step 4: Loading existing LoRA weights...")
model = PeftModel.from_pretrained(base_model, lora_path)
print("✅ Existing LoRA weights loaded.")

# --- Apply new LoRA config ---
print("🧩 Step 5: Applying new LoRA configuration for stacking...")
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ New LoRA configuration applied.")

# --- Load and preprocess dataset ---
print("📄 Step 6: Loading dataset...")
raw_dataset = load_dataset("json", data_files=os.path.join(dataset_path, dataset_file))["train"]
print(f"📊 Original dataset size: {len(raw_dataset)} samples")

# --- Tokenize dataset ---
print("✍ Step 7: Formatting and tokenizing...")

def format_alpaca(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]

    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"

    return {
        "input_ids": tokenizer(prompt + output_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")["input_ids"][0],
        "labels": tokenizer(output_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")["input_ids"][0]
    }

tokenized_dataset = raw_dataset.map(format_alpaca, num_proc=4)
print("✅ Tokenization complete.")

# --- Training ---
print("🚀 Step 8: Beginning training...")

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    logging_dir=f"{output_dir}/logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",
    save_total_limit=1,
    learning_rate=2e-4,
    fp16=True,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("🏁 Training in progress...")
trainer.train()
print("✅ Training complete.")

# --- Save Model ---
print("💾 Step 9: Saving new LoRA weights...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Final stacked LoRA model saved to: {output_dir}")

🔧 Starting continued LoRA fine-tuning script...
📁 Base Model: TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0
🔁 Continue LoRA fine-tuning from: /kaggle/input/finetuned/tinyllama-lora-finetuned
📂 Dataset path: /kaggle/input/medidata/second_half.json
💾 Output will be saved to: /kaggle/working/tinyllama-lora-finetuned-v2
📦 Step 1: Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Map (num_proc=4):   0%|          | 0/62734 [00:00<?, ? examples/s]

✅ Tokenization complete.
🚀 Step 8: Beginning training...
🏁 Training in progress...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-258cfb8a5e31>:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pytho

Step,Training Loss
10,10.905100
20,10.415700
30,9.072600
40,8.985200
50,8.900200
60,8.558300
70,8.728800
80,9.069100
90,9.038500
100,8.905800


✅ Training complete.
💾 Step 9: Saving new LoRA weights...
✅ Final stacked LoRA model saved to: /kaggle/working/tinyllama-lora-finetuned-v2
